In [21]:
import pandas as pd
import ast
qa_result_df = pd.read_csv("data/interim/qa_result_2023-07-19.csv")
# convert the 2nd column to dictionary by literal eval function and then convert to two columns
import re

import re

def correct_string_format(s):
    # Extract the title part from the string
    match = re.search(r'"title": (.+)}', s)
    if match is not None:  # check if match found
        start, end = match.span(1)
        # Remove existing double quotes
        removed_quotes_s = s[start:end].replace('"', '')
        # Put the value of "title" within quotes and replace the original in s
        corrected_s = s[:start] + '"' + removed_quotes_s + '"' + s[end:]
        return corrected_s
    else:
        return s  # return the original string if no match is found

qa_result_df['label_dict'] = qa_result_df.iloc[:,3].apply(correct_string_format)
qa_result_df["label_dict"] = qa_result_df['label_dict'].apply(lambda x: ast.literal_eval(x))
qa_result_df = qa_result_df.join(pd.json_normalize(qa_result_df["label_dict"]))
qa_result_df["summary"] = qa_result_df["summary"].apply(lambda x: ' '.join(x))

In [91]:
import pandas as pd
import ast
from geotext import GeoText
from geopy.geocoders import Nominatim
from tqdm import tqdm, notebook
def extract_location(text):
    # create a geocoder object
    geolocator = Nominatim(user_agent="my-app")
    location = geolocator.geocode(text)
    if location is not None:
        lat = location.latitude
        lon = location.longitude
        return lat, lon
    else:
        print("Got an error for ", text)    
        return None


qa_result_df = pd.read_csv("data/interim/qa_result_2023-07-19.csv").iloc[0:10,:]

qa_result_df["label_dict"] = qa_result_df.iloc[:,3].apply(lambda x: ast.literal_eval(x))
qa_result_df = qa_result_df.join(pd.json_normalize(qa_result_df["label_dict"]))
# set the first column as index
qa_result_df.set_index("file_name", inplace=True)
qa_result_df = qa_result_df["study_area"].explode().to_frame()
# create a string of "Country, City" for each paper
qa_result_df["study_area"] = qa_result_df["study_area"].apply(lambda x: ast.literal_eval(str(x)))
qa_result_df["study_area"] = qa_result_df["study_area"].apply(lambda x: x["Country"] + ", " + x["City"] if x.get("City") not in [None, "not mentioned", "not specified"] else x["Country"])
qa_result_df["study_area"] = qa_result_df["study_area"].apply(lambda x: extract_location(str(x)))
qa_result_df

,study_area
file_name,
10.1016_j.landurbplan.2022.104358.txt,"(31.2323437, 121.4691024)"
10.1016_j.jenvp.2023.102028.txt,"(39.084802, 117.1959904)"
10.1016_j.ufug.2022.127712.txt,"(60.1674881, 24.9427473)"
10.1016_j.ufug.2019.126477.txt,"(50.0874654, 14.4212535)"
10.1016_j.ufug.2020.126903.txt,"(31.2323437, 121.4691024)"
10.1016_j.buildenv.2021.108323.txt,"(29.8622194, 121.6203873)"
10.1016_j.ufug.2017.05.002.txt,"(53.9097123, -122.7623262)"
10.1016_j.cities.2022.103650.txt,"(52.2434979, 5.6343227)"
10.1016_j.jenvp.2019.03.005.txt,"(-34.135635300000004, -70.73998533381518)"


In [4]:
import pandas as pd
qa_result_df = pd.read_csv("data/interim/qa_result_2023-07-19.csv")
qa_result_df.iloc[:,6].unique()

array(['{"other_data_sources": "not applicable"}',
       '{\n  "other_data_sources": "not applicable"\n}',
       '{\n  "other_data_sources": {\n    "sound/auditory": "sound level meter",\n    "lighting": "illuminance meter"\n  }\n}',
       '{\n  "other_data_sources": {\n    "smell/olfactory": "not applicable",\n    "texture": "not applicable",\n    "sound/auditory": "not applicable"\n  }\n}',
       '{\n  "other_data_sources": {\n    "OTHER_SENSORY": "not applicable"\n  }\n}',
       '{\n  "other_data_sources": {\n    "Perceived oppressiveness": "not applicable"\n  }\n}',
       '{\n  "other_data_sources": {\n    "smell/olfactory": "not applicable",\n    "texture": "not applicable",\n    "sound/auditory": "https://orangefreesounds.com/sound-therapy-morning-birds/"\n  }\n}',
       '{\n  "other_data_sources": {\n    "OTHER_SENSORY": "DATA_SOURCE"\n  }\n}',
       '{\n  "other_data_sources": {\n    "sensory_data": "not applicable"\n  }\n}',
       '{\n  "other_data_sources": {\n    "s